# Giving arms and legs to agents with tools

An agent without any tool can only reason about the question it received. In order to allow agents to execute things we need to give them "arms and legs" using tools. Watch the great session [Scott and Mark learn AI](https://build.microsoft.com/en-US/sessions/10424a54-b809-48fc-9c8e-b8d4e3d0823a) on Microsoft Build 2024 to learn more about tools.

In Azure Agent Service there are some built-in tools, but we can also provide custom tools to interact with external systems.

First, let's setup the project.

In [ ]:
import os, jsonref
from pprint import pp as pp

from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import CodeInterpreterTool
from azure.ai.projects.models import FunctionTool, ToolSet
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails

# Load environment variables from .env file
load_dotenv()

def pprint(obj):
    pp(obj.as_dict() if hasattr(obj, "as_dict") else obj, width=100)

# Print the environments we will be using.
print(f"PROJECT_CONNECTION_STRING: {os.getenv('PROJECT_CONNECTION_STRING')}")
print(f"BING_CONNECTION_NAME: {os.getenv('BING_CONNECTION_NAME')}")  

Get the project reference using the current authenticated user and the connection string to the project where the agents will be created. 

In [2]:
project = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

## Let the agent write and execute Python code.

One of the built-in tools is the CodeInterpreter. This tool will allow the agent to write and execute Python code in a sandbox environment.

Let's instantiate the [CodeInterpreteTool](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/code-interpreter?tabs=python&pivots=overview) class and create an agent.

In [ ]:
# Create a code interpreter tool instance.
code_interpreter = CodeInterpreterTool()

statistics_agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="Statistics Expert",
    instructions="You are an expert on statistics, providing statistics help for users.",
    description="This agent was created to provide guidance for our users.",
    metadata= {
        "department": "finance",
        "owner": "jim"
    },
    
    # This is where assign the code interpreter to this agent.
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources
)
pprint(statistics_agent)

Let's create a thread to add the messages.

In [ ]:
thread_bill = project.agents.create_thread(
    metadata= {
        "entraUserId": "444dfd30-8420-4a8d-b155-4b5f05994545"
    }
)
pprint(thread_bill)

Now that we have a thread, let's add a message.

In [ ]:
message = project.agents.create_message(
    thread_id=thread_bill.id,
    role="user",
    content="Create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million",
)
pprint(message)

List all messages in the thread.

In [ ]:
pprint(project.agents.list_messages(thread_bill.id))

Now it's time to let the agent write and run some code!

In [ ]:
pprint(project.agents.create_and_process_run(thread_id=thread_bill.id, assistant_id=statistics_agent.id))

Let's show the message created by the code interpreter in the thread.

In [ ]:
pprint(project.agents.list_messages(thread_bill.id))

## Using a custom function

It's possible to define a function in Python and let the agent use it using a feature called **function calling**.

The function below uses mock data to return weather predictions. Once called, it will return the weather in the location specified.

In order to extract the location from the user question the model will be used, and will return to our application the name and parameters of the function to be executed.
Once the function is executed our application will return to the agent the result to finish the execution of the user question and provide the final answer.

In [27]:
import json
from typing import Any, Callable, Set

def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """

    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {
        "New York": "Sunny, 25°C",
        "London": "Cloudy, 18°C", 
        "Tokyo": "Rainy, 22°C"
    }
    
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    
    weather_json = json.dumps({"weather": weather})
    return weather_json

user_functions: Set[Callable[..., Any]] = {
    fetch_weather,
}

functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

Now we will create an agent to help users query about the weather. We will also add the  custom tool we implemented in Python.

In [ ]:
weather_agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="Weather Expert",
    instructions="You are an expert on weather, providing predictions to users.",
    
    # Assign our custom function to the agent.
    toolset=toolset
)
pprint(weather_agent)

Let's a create a thread and a message.

In [ ]:
thread_john = project.agents.create_thread()

message_john = project.agents.create_message(
    thread_id=thread_john.id,
    role="user",
    content="Hello, send an email with the datetime and weather information in New York?",
)
pprint(project.agents.list_messages(thread_john.id))

Execute the agent.

In [ ]:
run = project.agents.create_and_process_run(thread_id=thread_john.id, assistant_id=weather_agent.id)
pprint(run)

Let's see what our weather agent did. In this case, as we provided a custom function, the SDK automatically called it with the right parameters and our custom Python function was called.

As you can see the agent created the correct answer based in the mock data, where the weather in New York is sunny, 25°C.

In [ ]:
pprint(project.agents.list_messages(thread_john.id))

## Using OpenAPI to call functions

Another way to provide functions to an agent is using the [OpenAPI](https://swagger.io/specification/) specification.

Let's see how it works. First we need to load the specification file, and a create an OpenAPI tool from it. Here we are reading the specification from a file, but it could be an URL.


In [14]:
with open('./order-api.json', 'r') as f:
    order_api_spec = jsonref.loads(f.read())

# Create Auth object for the OpenApiTool (note that connection or managed identity auth setup requires additional setup in Azure)
auth = OpenApiAnonymousAuthDetails()

# Initialize agent OpenApi tool using the read in OpenAPI spec
openapi = OpenApiTool(
    name="order_api", 
    spec=order_api_spec,
    description="Order API used to get and create orders.", 
    auth=auth
)

An OpenAPI specification might have hundreds of operations where each operation might get, create, update or delete data, or trigger actions.

In order to use the operations they need to be very well documented using natural language, because that's what the model of the agent will see, and will use to evaluate what functions it need to execute in order to generate an answer.

Let's print the OpenAPI specification we read.

In [ ]:
openapi.definitions

Now that we loaded the OpenAPI spec, let's create an agent pointing to it.

In [ ]:
order_agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="Order Assistant",
    instructions="You know everything about orders.",
    
    # This is where we assign the OpenAPI tool to the agent.
    tools=openapi.definitions
)
pprint(order_agent)

The same way we did previously, now we need a thread with a message to execute the agent. The code below will create a thread with a single message and execute it. It's execution will make an API call to the API we loaded previously. In this case it will call the _/orders/123_ endpoint to get detailed information about this order.

If you want to take a look in the source code behind the API you can find it in the file **order-api-azure-function.py**, which is a shared Azure Function that was deployed for this workshop.

In [ ]:
thread_alice = project.agents.create_thread()

message_alice = project.agents.create_message(
    thread_id=thread_alice.id,
    role="user",
    content="What is the price of order number 123?",
)

run = project.agents.create_and_process_run(
    thread_id=thread_alice.id, 
    assistant_id=order_agent.id
)

pprint(project.agents.list_messages(thread_alice.id))

During this lab we showed that it's possible to give tools to agents, and they can use these tools to execute actions. This way we are giving more power to agents, and they cannot only reason, but they can actively act to accomplish a goal.

In addition to the code interpreter, custom function and OpenAPI tool we also have a tool that allow **Azure Functions** to be used. You can find more information [here](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-functions?pivots=overview).

You have reached the end of this lab. 👏